# Computing FSAs

**(C) 2017-2019 by [Damir Cavar](http://damir.cavar.me/)**

**Version:** 1.0, September 2019

**License:** [Creative Commons Attribution-ShareAlike 4.0 International License](https://creativecommons.org/licenses/by-sa/4.0/) ([CA BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/))

Consider the following automaton:

<img src="NDFSAMatrixOp.png" caption="Non-deterministic Finite State Automaton" style="width: 200px;"/>

We can represent it in terms of transition tables. We will use the Python numpy module for that.

In [1]:
from numpy import array

The transitions are coded in terms of state to state transitions. The columns and rows represent the states 0, 1, and 2. The following transition matrix shows all transitions that are associated with the label "a", that is from 0 to 0, from 0 to 1, and from 1 to 0.

In [2]:
a = array([
    [1, 1, 0],
    [1, 0, 0],
    [0, 0, 0]
])

The following transition matrix shows that for the transitions associated with "b".

In [3]:
b = array([
    [0, 1, 0],
    [0, 1, 0],
    [0, 0, 0]
])

The following transition matrix shows this for the transitions associated with "c".

In [4]:
c = array([
    [0, 0, 0],
    [0, 0, 1],
    [0, 0, 0]
])

We can define the start state using an init vector. This init vector indicates that the start state should be 0.

In [6]:
init = array([
    [1, 0, 0]
])

The set of final states can be encoded as a column vector that in this case defines state 3 as the only final state.

In [7]:
final = array([
    [0],
    [0],
    [1]
])

If we want to compute the possibility for a sequence like "aa" to be accepted by this automaton, we could compute the dot product of the init-vector and the a matrices, with the dot product of the final state.

In [19]:
init.dot(a).dot(a).dot(final)

array([[0]])

The 0 indicates that there is no path from the initial state to the final state based on a sequence "aa".

Let us verify this for a sequence "bc", for which we know that there is such a path:

In [20]:
init.dot(b).dot(c).dot(final)

array([[1]])

Just to verify once more, let us consider the sequence "aab":

In [21]:
init.dot(a).dot(a).dot(b).dot(final)

array([[0]])

**(C) 2016-2019 by [Damir Cavar](http://damir.cavar.me/) <<dcavar@iu.edu>>**